# Part I. ETL Pipeline for Pre-Processing the Files

### This notebook connect to the Cassandra running in Docker and create a keyspace called sparkify. The modeling process follow this 3 queries:
1. `artist`, `song`(is the name/title of it) and song's `length` (aka durattion) for `sessionId=338` and `itemInSession=4`
2. `artist`, `song` but sorted by `itemInSession` and user `firstName` and `lastName` for `userId=10` and `sessionId=182`
3. And all users `firstName` and `lastName` for the song *'All Hands Against His Own'*

#### Import Python packages 

In [1]:
import pandas as pd
from cassandra.cluster import Cluster
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
print(os.getcwd())

filepath = os.getcwd() + '/event_data'

for root, dirs, files in os.walk(filepath):
    file_path_list = glob.glob(os.path.join(root,'*'))


/home/jovyan


#### Reading all data from event_data and creating a single file to work on output_data

In [3]:
full_data_rows_list = [] 
    
for f in file_path_list:

    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('./output_data/event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


#### Check the number of rows in combined events csv file.

In [4]:
with open('./output_data/event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


#### Create cluster and session connection.

In [6]:
try: 
    cluster = Cluster(['cassandra'])
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [8]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [10]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

#### Create the `songinfo_by_session_by_item` table 

In [13]:
query = "CREATE TABLE IF NOT EXISTS songinfo_by_session_by_item "
query = query + " (sessionId INT, itemInSession INT, artist TEXT, song TEXT, length FLOAT, PRIMARY KEY (sessionId, itemInSession));"

try:
    session.execute(query)
except Exception as e:
    print(e)

### Query 1

1. `artist`, `song`(is the name/title of it) and song's `length` (aka durattion) for `sessionId=338` and `itemInSession=4`

##### `songinfo_by_session_by_item` includes artist, song title and song length information for a given `sessionId` and `itemInSessionId`.

##### The table is partition and clusterized by `sessionId` and `itemInSessionId` to group the data and to have good performance on responses.
##### The `sessionId` as PARTITION_KEY and the `itemInSessionId` as cluster column to sort the result.

In [14]:
file = './output_data/event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    
    for line in csvreader:
        query = "INSERT INTO songinfo_by_session_by_item (sessionId, itemInSession, artist, song, length)"
        query = query + " VALUES (%s, %s, %s, %s, %s);"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [15]:
query = "SELECT artist, song, length FROM songinfo_by_session_by_item WHERE sessionId=338 AND itemInSession=4;"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

to_df = [{"artist": row.artist, "song": row.song, "length": row.length} for row in rows]

pd.DataFrame(to_df)

,artist,song,length
0,Faithless,Music Matters (Mark Knight Dub),495.307312


#### Create the `songinfo_by_user_by_session` table 

In [16]:
query = "CREATE TABLE IF NOT EXISTS songinfo_by_user_by_session "
query = query + " (userId INT, sessionId INT, itemInSession INT, artist TEXT, song TEXT, firstName TEXT, lastName TEXT, PRIMARY KEY ((userId, sessionId), itemInSession));"

try:
    session.execute(query)
except Exception as e:
    print(e)

## Query 2

##### `songinfo_by_user_by_session` includes artist, song and user for a given `userId` and `sessionId`

##### The table is partition and clusterized by `userId` and `sessionId` to group the data and to have good performance on responses.
##### The `userId` as PARTITION_KEY and the `sessionId` as cluster column to sort the result.

In [17]:
file = './output_data/event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    
    for line in csvreader:
        query = "INSERT INTO songinfo_by_user_by_session (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + " VALUES (%s, %s, %s, %s, %s, %s, %s);"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

In [18]:
query = "SELECT artist, song, firstName, lastName FROM songinfo_by_user_by_session WHERE userId=10 AND sessionId=182 ORDER BY itemInSession;"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

to_df = [{"artist": row.artist, "song": row.song, "firstName": row.firstname, "lastName": row.lastname} for row in rows]

pd.DataFrame(to_df)

,artist,song,firstName,lastName
0,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz
1,Three Drives,Greece 2000,Sylvie,Cruz
2,Sebastien Tellier,Kilometer,Sylvie,Cruz
3,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz


#### Create the `userinfo_by_song` table 

In [19]:
query = "CREATE TABLE IF NOT EXISTS userinfo_by_song "
query = query + " (song TEXT, firstName TEXT, lastName TEXT, userId INT, PRIMARY KEY (song, userId));"

try:
    session.execute(query)
except Exception as e:
    print(e)

## Query 3

##### `userinfo_by_song` includes user names for a given song.

##### The table is partition and clusterized by `song` and `userId` to group the data and to have good performance on responses.
##### The `song` as PARTITION_KEY and the `userId` as cluster column to sort the result.
##### We are searching by `song` so do the `userId` as a cluster column will help to make the result more organazide.

In [20]:
file = './output_data/event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    
    for line in csvreader:
        query = "INSERT INTO userinfo_by_song (song, firstName, lastName, userId)"
        query = query + " VALUES (%s, %s, %s, %s);"
        session.execute(query, (line[9], line[1], line[4], int(line[10])))

In [21]:
query = "SELECT firstName, lastName FROM userinfo_by_song WHERE song='All Hands Against His Own';"

try:
    rows = session.execute(query)
except Exception as e:
    print(e)

to_df = [{"firstname": row.firstname, "lastname": row.lastname} for row in rows]

pd.DataFrame(to_df)

,firstname,lastname
0,Jacqueline,Lynch
1,Tegan,Levine
2,Sara,Johnson


### Drop the tables before closing out the sessions

In [22]:
query = "drop table if exists songinfo_by_session_by_item"
query1 = "drop table if exists songinfo_by_user_by_session"
query2 = "drop table if exists userinfo_by_song"
try:
    session.execute(query)
except Exception as e:
    print(e)
try:
    session.execute(query1)
except Exception as e:
    print(e)
try:
    session.execute(query2)
except Exception as e:
    print(e)

### Close the session and cluster connection

In [23]:
session.shutdown()
cluster.shutdown()